In [26]:
import tensorflow as tf
import tensorflow_hub as hub
import os
from typing import Tuple

In [27]:
train_dir = 'dataset/train'
test_dir = 'dataset/test'

In [62]:
BATCH_SIZE = 18
IMG_HEIGHT = 144
IMG_WIDTH = 144

In [64]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2, rescale=1./255,rotation_range=60)

train_data = train_datagen.flow_from_directory(
        train_dir,  shuffle=True, color_mode="rgb", class_mode="categorical",subset='training',
        target_size=(IMG_WIDTH,IMG_HEIGHT), batch_size=BATCH_SIZE)
    
validation_data = train_datagen.flow_from_directory( train_dir,
        shuffle=True, color_mode="rgb", class_mode="categorical",subset='validation',
        target_size=(IMG_WIDTH,IMG_HEIGHT), batch_size=BATCH_SIZE)


Found 24525 images belonging to 25 classes.
Found 6122 images belonging to 25 classes.
Found 250 images belonging to 25 classes.


In [79]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,rotation_range=60)

test_data = valid_datagen.flow_from_directory(test_dir, shuffle=False, color_mode="rgb", class_mode="categorical",
        target_size=(IMG_WIDTH,IMG_HEIGHT))

Found 250 images belonging to 25 classes.


In [65]:
INCEPTIONV3_TFHUB = 'https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4'

In [66]:
input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)
num_classes = 25

# fetch the feature extractor from the tf_hub
feature_extractor = hub.KerasLayer(INCEPTIONV3_TFHUB, input_shape=input_shape)

# make the feature extractor trainable
feature_extractor.trainable = True

# create the sequential model
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
])

In [68]:
# print the summary of the model
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_6 (KerasLayer)  (None, 2048)              21802784  
                                                                 
 flatten_6 (Flatten)         (None, 2048)              0         
                                                                 
 dense_9 (Dense)             (None, 25)                51225     
                                                                 
Total params: 21,854,009
Trainable params: 21,819,577
Non-trainable params: 34,432
_________________________________________________________________


In [69]:
# compile the model
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [70]:
# train the model
history = model.fit(
    train_data,
    epochs=2,
    steps_per_epoch=train_data.samples//train_data.batch_size,
    validation_data=validation_data,
)

Epoch 1/2
1362/1362 [==============================] - 162s 113ms/step - loss: 0.7788 - accuracy: 0.8406 - val_loss: 0.4971 - val_accuracy: 0.9198
Epoch 2/2
1362/1362 [==============================] - 148s 109ms/step - loss: 0.4474 - accuracy: 0.9380 - val_loss: 0.3573 - val_accuracy: 0.9685


In [71]:
# evaluate the model
loss, accuracy = model.evaluate(test_data)
# train accuracy
# train_loss, train_accuracy = model.evaluate(train_data)

8/8 [==============================] - 3s 243ms/step - loss: 0.3710 - accuracy: 0.9720


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [98]:
model.save("models/20220405_1907")

INFO:tensorflow:Assets written to: models/20220405_1907\assets


INFO:tensorflow:Assets written to: models/20220405_1907\assets


In [101]:
json_path = "models/final_model.json"
h5_path = "models/final_model.h5"
# serialize model to JSON
model_json = model.to_json()
with open(json_path, "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(h5_path)

In [104]:
# load json and create model
json_file = open(json_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json, custom_objects={'KerasLayer':hub.KerasLayer})
# load weights into new model
loaded_model.load_weights(h5_path)
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
score = loaded_model.evaluate(test_data, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 96.40%


In [74]:
import matplotlib.pyplot as plt
import numpy as np

In [80]:
predictions = model.predict(test_data)

In [83]:
y_pred= np.argmax(predictions, axis=1)

In [84]:
y_true = test_data.labels

In [89]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_true, y_pred, labels=range(0,25))
matrix.diagonal()

array([10,  9, 10, 10,  8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  9,  9,
       10,  8, 10,  9,  9, 10,  9, 10], dtype=int64)

In [90]:
from sklearn.metrics import precision_recall_fscore_support as score

precision, recall, fscore, support = score(y_true, y_pred)


precision: [0.90909091 1.         1.         1.         1.         1.
 1.         0.83333333 1.         1.         1.         1.
 1.         1.         1.         0.9        0.69230769 1.
 0.88888889 1.         1.         0.9        1.         1.
 1.        ]
recall: [1.  0.9 1.  1.  0.8 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.9 0.9 1.
 0.8 1.  0.9 0.9 1.  0.9 1. ]
fscore: [0.95238095 0.94736842 1.         1.         0.88888889 1.
 1.         0.90909091 1.         1.         1.         1.
 1.         1.         1.         0.9        0.7826087  1.
 0.84210526 1.         0.94736842 0.9        1.         0.94736842
 1.        ]
support: [10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10]


In [91]:
classes= ["Apple Black Rot", "Apple Cedar Apple Rust", "Apple Healthy", "Apple Scab", "Corn Maize Cercospora Leaf Spot Gray Leaf Spot", "Corn Maize Common Rust", "Corn Maize Healthy", "Corn Maize Northern Leaf Blight", "Grape Black Rot", "Grape Esca Black Measles", "Grape Healthy", "Grape Leaf Blight Isariopsis Leaf Spot", "Potato Early Blight", "Potato Healthy", "Potato Late Blight", "Tomato Bacterial Spot", "Tomato Early Blight", "Tomato Healthy", "Tomato Late Blight", "Tomato Leaf Mold", "Tomato Mosaic Virus", "Tomato Septoria Leaf Spot", "Tomato Spider Mites Two-Spotted Spider Mite", "Tomato Target Spot", "Tomato Yellow Leaf Curl Virus"]

In [92]:
import pandas as pd

In [94]:
metrics_table = pd.DataFrame(zip(classes,precision,recall,fscore,support),columns=["Class","Precision","Recall","Fscore","Support"])

,Class,Precision,Recall,Fscore,Support
0,Apple Black Rot,0.909091,1.0,0.952381,10
1,Apple Cedar Apple Rust,1.000000,0.9,0.947368,10
2,Apple Healthy,1.000000,1.0,1.000000,10
3,Apple Scab,1.000000,1.0,1.000000,10
4,Corn Maize Cercospora Leaf Spot Gray Leaf Spot,1.000000,0.8,0.888889,10
5,Corn Maize Common Rust,1.000000,1.0,1.000000,10
6,Corn Maize Healthy,1.000000,1.0,1.000000,10
7,Corn Maize Northern Leaf Blight,0.833333,1.0,0.909091,10
8,Grape Black Rot,1.000000,1.0,1.000000,10
9,Grape Esca Black Measles,1.000000,1.0,1.000000,10


In [95]:
metrics_table.to_csv("test_metrics.csv",index=False)